In [3]:
import pandas as pd
import math
from itertools import combinations

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = math.sin(dLat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dLon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def compute_mst(points):
    n = len(points)
    edges = []
    for i in range(n):
        for j in range(i + 1, n):
            dist = haversine(points[i][0], points[i][1], points[j][0], points[j][1])
            edges.append((dist, i, j))
    edges.sort()
    parent = list(range(n))
    def find(u):
        while parent[u] != u:
            parent[u] = parent[parent[u]]
            u = parent[u]
        return u
    def union(u, v):
        u_root = find(u)
        v_root = find(v)
        if u_root != v_root:
            parent[v_root] = u_root
    mst_sum = 0
    mst_edges = []
    for edge in edges:
        dist, u, v = edge
        if find(u) != find(v):
            union(u, v)
            mst_sum += dist
            mst_edges.append(edge)
            if len(mst_edges) == n - 1:
                break
    return mst_sum

def convert_time_to_minutes(time_str):
    return int(time_str.split(':')[0]) * 60 + int(time_str.split(':')[1])

def tsp_nearest_neighbor(points):
    n = len(points)
    visited = [False] * n
    path = [0]
    visited[0] = True
    current = 0
    for _ in range(n - 1):
        nearest_dist = float('inf')
        nearest_idx = -1
        for i in range(n):
            if not visited[i]:
                dist = haversine(points[current][0], points[current][1], points[i][0], points[i][1])
                if dist < nearest_dist:
                    nearest_dist = dist
                    nearest_idx = i
        if nearest_idx == -1:
            break
        path.append(nearest_idx)
        visited[nearest_idx] = True
        current = nearest_idx
    path.append(0)
    return path

# Load data
store_df = pd.read_excel('Data/SmartRoute Optimizer.xlsx', sheet_name='Store Location')
print(store_df)
store_lat = store_df['Latitute']
store_lon = store_df['Longitude']

shipments_df = pd.read_excel('Data/SmartRoute Optimizer.xlsx', sheet_name='Shipments_Data')
shipments = []
for _, row in shipments_df.iterrows():
    start, end = row['Delivery Timeslot'].split('-')
    shipments.append({
        'id': row['Shipment ID'],
        'lat': row['Latitude'],
        'lon': row['Longitude'],
        'start': convert_time_to_minutes(start.strip()),
        'end': convert_time_to_minutes(end.strip())
    })

# Priority vehicles (3W, 4W-EV) and regular 4W
vehicles = [
    {'type': '3W', 'remaining': 50, 'capacity': 5, 'max_radius': 15},
    {'type': '4W-EV', 'remaining': 25, 'capacity': 8, 'max_radius': 20},
    {'type': '4W', 'remaining': float('inf'), 'capacity': 25, 'max_radius': float('inf')}
]

trips = []

# Group shipments into trips
for shipment in sorted(shipments, key=lambda x: x['start']):
    added = False
    for trip in trips:
        # Check if the shipment can be added to the trip
        new_start = min(trip['start'], shipment['start'])
        new_end = max(trip['end'], shipment['end'])
        available_time = new_end - new_start
        new_shipments = trip['shipments'] + [shipment]
        points = [(store_lat, store_lon)] + [(s['lat'], s['lon']) for s in new_shipments]
        mst_dist = compute_mst(points)
        for vehicle in vehicles:
            if vehicle['remaining'] <= 0 or vehicle['type'] == '4W':
                continue
            if len(new_shipments) <= vehicle['capacity'] and mst_dist <= vehicle['max_radius']:
                trip_time = (mst_dist * 5) + (len(new_shipments) * 10)
                if trip_time <= available_time:
                    trip['shipments'] = new_shipments
                    trip['start'] = new_start
                    trip['end'] = new_end
                    trip['mst_dist'] = mst_dist
                    trip['vehicle'] = vehicle['type']
                    vehicle['remaining'] -= 1
                    added = True
                    break
        if added:
            break
    if not added:
        # Create a new trip with the shipment
        points = [(store_lat, store_lon), (shipment['lat'], shipment['lon'])]
        mst_dist = compute_mst(points)
        assigned_vehicle = None
        for vehicle in vehicles:
            if vehicle['remaining'] <= 0:
                continue
            if len([shipment]) <= vehicle['capacity'] and mst_dist <= vehicle['max_radius']:
                assigned_vehicle = vehicle
                vehicle['remaining'] -= 1
                break
        if not assigned_vehicle:
            assigned_vehicle = vehicles[2]  # 4W
        trips.append({
            'shipments': [shipment],
            'start': shipment['start'],
            'end': shipment['end'],
            'mst_dist': mst_dist,
            'vehicle': assigned_vehicle['type'],
            'vehicle_capacity': assigned_vehicle['capacity'],
            'vehicle_max_radius': assigned_vehicle['max_radius']
        })

# Reassign shipments to ensure 50% capacity utilization
for trip in trips:
    if len(trip['shipments']) < trip['vehicle_capacity'] * 0.5:
        # Try to merge this trip with another trip
        for other_trip in trips:
            if other_trip == trip:
                continue
            combined_shipments = trip['shipments'] + other_trip['shipments']
            if len(combined_shipments) <= trip['vehicle_capacity']:
                points = [(store_lat, store_lon)] + [(s['lat'], s['lon']) for s in combined_shipments]
                mst_dist = compute_mst(points)
                if mst_dist <= trip['vehicle_max_radius']:
                    trip['shipments'] = combined_shipments
                    trip['mst_dist'] = mst_dist
                    trips.remove(other_trip)
                    break

# Sequence shipments in each trip using TSP
for trip in trips:
    points = [(store_lat, store_lon)] + [(s['lat'], s['lon']) for s in trip['shipments']]
    path = tsp_nearest_neighbor(points)
    shipment_indices = path[1:-1]  # exclude store at start and end
    ordered_shipments = [trip['shipments'][i-1] for i in shipment_indices]
    trip['shipments'] = ordered_shipments

# Generate output
output = []
for idx, trip in enumerate(trips):
    trip_time = (trip['mst_dist'] * 5) + (len(trip['shipments']) * 10)
    available_time = trip['end'] - trip['start']
    time_uti = trip_time / available_time if available_time != 0 else 0
    capacity_uti = len(trip['shipments']) / trip['vehicle_capacity']
    cov_uti = trip['mst_dist'] / trip['vehicle_max_radius'] if trip['vehicle_max_radius'] != float('inf') else 0
    output.append({
        'Trip_ID': idx + 1,
        'Shipment_Sequence': [s['id'] for s in trip['shipments']],
        'Vehicle_Type': trip['vehicle'],
        'MST_DIST': round(trip['mst_dist'], 2),
        'TRIP_TIME': round(trip_time, 2),
        'CAPACITY_UTI': round(capacity_uti, 2),
        'TIME_UTI': round(time_uti, 2),
        'COV_UTI': round(cov_uti, 2) if isinstance(cov_uti, float) else 'N/A'
    })

# Convert to DataFrame
output_df = pd.DataFrame(output)
print(output_df.to_string(index=False))

    Latitute  Longitude
0  19.075887  72.877911


/tmp/ipykernel_26036/2314204994.py:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  dLat = math.radians(lat2 - lat1)
/tmp/ipykernel_26036/2314204994.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  dLon = math.radians(lon2 - lon1)
/tmp/ipykernel_26036/2314204994.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a = math.sin(dLat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dLon/2)**2


 Trip_ID                                      Shipment_Sequence Vehicle_Type  MST_DIST  TRIP_TIME  CAPACITY_UTI  TIME_UTI COV_UTI
       1                                       [372, 1019, 577]           4W     24.95     154.76          0.12      1.03     N/A
       2                                                  [375]           4W     13.81      79.05          0.04      0.53     N/A
       3                                            [376, 1020]           4W      8.69      63.47          0.08      0.42     N/A
       4                                                  [377]           4W      5.39      36.96          0.04      0.25     N/A
       5                             [383, 1025, 219, 579, 326]           4W     28.75     193.77          0.20      1.29     N/A
       6                                                  [385]           4W      5.34      36.68          0.04      0.24     N/A
       7                                            [1026, 386]           4W      8.30    